## This notebook demonstrates bundles to seqex example

Install packages required for apache-beam and protobuf

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

from absl import app
from absl import flags
import apache_beam as beam
from google.protobuf import text_format
from proto.stu3 import google_extensions_pb2
from proto.stu3 import resources_pb2
from proto.stu3 import version_config_pb2
from tensorflow.core.example import example_pb2
from py.google.fhir.seqex import bundle_to_seqex

/usr/local/envs/py2env/lib/python2.7/site-packages/ipykernel/__main__.py:12: ImportWarning: Not importing directory '/usr/local/fhir/proto': missing __init__.py
/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._conv import register_converters as _register_converters
/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/_hl/group.py:22: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .. import h5g, h5i, h5o, h5r, h5t, h5l, h5p
/usr/local/envs/py2env/lib/python2.7/site-packages/scipy/ndimage

In [2]:
def _get_version_config(version_config_path):
  with open(version_config_path) as f:
    return text_format.Parse(f.read(), version_config_pb2.VersionConfig())

# Initialize variables

In [3]:
from apache_beam.options.pipeline_options import GoogleCloudOptions
from apache_beam.options.pipeline_options import StandardOptions
from apache_beam.options.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import PipelineOptions

from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText
from apache_beam.metrics import Metrics
from apache_beam.metrics.metric import MetricsFilter

import apache_beam as beam
import re

options = PipelineOptions()
google_cloud_options = options.view_as(GoogleCloudOptions)
google_cloud_options.project = 'de-test-224618'
google_cloud_options.job_name = 'job1'
google_cloud_options.staging_location = 'gs://healthedatalab/staging'
google_cloud_options.temp_location = 'gs://healthedatalab/temp'
options.view_as(StandardOptions).runner = 'DirectRunner'


In [6]:
input_path = 'gs://healthedatalab/synthea/bundles/bundles*'
label_path = 'gs://healthedatalab/synthea/labels/label*'
output_path = 'gs://healthedatalab/synthea/seqex/seqex'

## Transform definition

In [7]:
p = beam.Pipeline(options=options)
#version_config = _get_version_config("gs://seqex/testdata/version_config.textproto")
version_config = _get_version_config("/usr/local/fhir/proto/stu3/version_config.textproto")

keyed_bundles = ( 
    p 
    | 'readBundles' >> beam.io.ReadFromTFRecord(
        input_path, coder=beam.coders.ProtoCoder(resources_pb2.Bundle))
    | 'KeyBundlesByPatientId' >> beam.ParDo(
        bundle_to_seqex.KeyBundleByPatientIdFn()))
event_labels = ( 
    p | 'readEventLabels' >> beam.io.ReadFromTFRecord(
        label_path,
        coder=beam.coders.ProtoCoder(google_extensions_pb2.EventLabel)))
keyed_event_labels = bundle_to_seqex.CreateTriggerLabelsPairLists(
    event_labels)
bundles_and_labels = bundle_to_seqex.CreateBundleAndLabels(
    keyed_bundles, keyed_event_labels)
_ = ( 
    bundles_and_labels
    | 'Reshuffle1' >> beam.Reshuffle()
    | 'GenerateSeqex' >> beam.ParDo(
        bundle_to_seqex.BundleAndLabelsToSeqexDoFn(
            version_config=version_config,
            enable_attribution=False,
            generate_sequence_label=False))
    | 'Reshuffle2' >> beam.Reshuffle()
    | 'WriteSeqex' >> beam.io.WriteToTFRecord(
        output_path,
        coder=beam.coders.ProtoCoder(example_pb2.SequenceExample)))


I0115 17:03:25.187494 140220694857472 gcsio.py:446] Starting the size estimation of the input
I0115 17:03:25.190587 140220694857472 client.py:614] Attempting refresh to obtain initial access_token
I0115 17:03:25.299873 140220694857472 gcsio.py:460] Finished listing 9 files in 0.112375020981 seconds.
I0115 17:03:25.321052 140220694857472 gcsio.py:446] Starting the size estimation of the input
I0115 17:03:25.324073 140220694857472 client.py:614] Attempting refresh to obtain initial access_token
I0115 17:03:25.409007 140220694857472 gcsio.py:460] Finished listing 1 files in 0.0879609584808 seconds.


Run the transform

In [8]:
   p.run().wait_until_finish()

I0115 17:03:28.478152 140220694857472 fn_api_runner.py:844] ==================== <function annotate_downstream_side_inputs at 0x7f876a3e4b90> ====================
I0115 17:03:28.480693 140220694857472 fn_api_runner.py:844] ==================== <function fix_side_input_pcoll_coders at 0x7f876a3e4de8> ====================
I0115 17:03:28.484848 140220694857472 fn_api_runner.py:844] ==================== <function lift_combiners at 0x7f876a3e4c80> ====================
I0115 17:03:28.487267 140220694857472 fn_api_runner.py:844] ==================== <function expand_gbk at 0x7f876a3e4c08> ====================
I0115 17:03:28.493345 140220694857472 fn_api_runner.py:844] ==================== <function sink_flattens at 0x7f876a3e4938> ====================
I0115 17:03:28.495371 140220694857472 fn_api_runner.py:844] ==================== <function greedily_fuse at 0x7f876a3e47d0> ====================
I0115 17:03:28.499965 140220694857472 fn_api_runner.py:844] ==================== <function sort_stag

I0115 17:04:12.200643 140220694857472 bundle_processor.py:303] finish <DoOperation CreateTriggerLabelsPairLists output_tags=['out'], receivers=[ConsumerSet[CreateTriggerLabelsPairLists.out0, coder=WindowedValueCoder[TupleCoder[BytesCoder, FastPrimitivesCoder]], len(consumers)=1]]>
I0115 17:04:12.202689 140220694857472 bundle_processor.py:303] finish <DoOperation GroupBundleAndTriggers/pair_with_trigger_labels_pair_lists output_tags=['out'], receivers=[ConsumerSet[GroupBundleAndTriggers/pair_with_trigger_labels_pair_lists.out0, coder=WindowedValueCoder[TupleCoder[LengthPrefixCoder[FastPrimitivesCoder], LengthPrefixCoder[FastPrimitivesCoder]]], len(consumers)=1]]>
I0115 17:04:12.205409 140220694857472 bundle_processor.py:303] finish <FlattenOperation GroupBundleAndTriggers/Flatten/Transcode/1 receivers=[ConsumerSet[GroupBundleAndTriggers/Flatten/Transcode/1.out0, coder=WindowedValueCoder[TupleCoder[LengthPrefixCoder[FastPrimitivesCoder], LengthPrefixCoder[FastPrimitivesCoder]]], len(cons

I0115 17:07:40.954925 140220694857472 bundle_processor.py:303] finish <DoOperation GenerateSeqex output_tags=['out'], receivers=[ConsumerSet[GenerateSeqex.out0, coder=WindowedValueCoder[ProtoCoder], len(consumers)=1]]>
I0115 17:07:40.956585 140220694857472 bundle_processor.py:303] finish <DoOperation Reshuffle2/AddRandomKeys output_tags=['out'], receivers=[ConsumerSet[Reshuffle2/AddRandomKeys.out0, coder=WindowedValueCoder[TupleCoder[FastPrimitivesCoder, ProtoCoder]], len(consumers)=1]]>
I0115 17:07:40.959649 140220694857472 bundle_processor.py:303] finish <DoOperation Reshuffle2/ReshufflePerKey/Map(reify_timestamps) output_tags=['out'], receivers=[ConsumerSet[Reshuffle2/ReshufflePerKey/Map(reify_timestamps).out0, coder=WindowedValueCoder[TupleCoder[LengthPrefixCoder[FastPrimitivesCoder], TupleCoder[LengthPrefixCoder[ProtoCoder], LengthPrefixCoder[FastPrimitivesCoder]]]], len(consumers)=1]]>
I0115 17:07:40.961956 140220694857472 bundle_processor.py:303] finish <DataOutputOperation Resh

I0115 17:08:21.180704 140220694857472 bundle_processor.py:303] finish <DataOutputOperation ref_PCollection_PCollection_40/Write >
I0115 17:08:21.187416 140220694857472 fn_api_runner.py:912] Running ((ref_PCollection_PCollection_32/Read)+(ref_AppliedPTransform_WriteSeqex/Write/WriteImpl/PreFinalize_58))+(ref_PCollection_PCollection_41/Write)
I0115 17:08:21.200042 140220694857472 bundle_processor.py:291] start <DataOutputOperation ref_PCollection_PCollection_41/Write >
I0115 17:08:21.202373 140220694857472 bundle_processor.py:291] start <DoOperation WriteSeqex/Write/WriteImpl/PreFinalize output_tags=['out']>
I0115 17:08:21.204864 140220694857472 bundle_processor.py:291] start <DataInputOperation ref_PCollection_PCollection_32/Read receivers=[ConsumerSet[ref_PCollection_PCollection_32/Read.out0, coder=WindowedValueCoder[FastPrimitivesCoder], len(consumers)=1]]>
I0115 17:08:21.209556 140220694857472 gcsio.py:446] Starting the size estimation of the input
I0115 17:08:21.211824 1402206948574

'DONE'

YAY!!